In [ ]:
# Write a function called optunamiser that uses the library optuna to optimise the degrees of a 
#function that creates a scikit learn scorer object from a polynomial quadratic function.
#The function should take as input an object called operator. It should create a function called objective, which should
# use trial.suggest_int to get three values - first_degree, second_degree and third_degree - from the argument params_grid
# It should then  create a piecewise quadratic function with numpy.piecewise, which is used to create a scikit-learn scorer object. 
# The scorer is then used to calculate the mean squared error of the .scoring method of the operator, which takes as input the scorer object. 
# The average of the "tpot_error" column of the error_df attribute of the operator is returned.
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import optuna


def optunamiser(operator):
    def objective(trial):
        first_degree = trial.suggest_int('first_degree', 0, 10)
        second_degree = trial.suggest_int('second_degree', 0, 10)
        third_degree = trial.suggest_int('third_degree', 0, 10)
        def piecewise_quadratic(x):
            return np.piecewise(x, [x < 0, x >= 0], [lambda x: x**2, lambda x: x**2])
        scorer = make_scorer(mean_squared_error, greater_is_better=False)
        operator.scorer = scorer
        operator.params_grid = {'first_degree': [first_degree], 'second_degree': [second_degree], 'third_degree': [third_degree]}
        operator.scoring(scorer = scorer)
        return operator.error_df['tpot_error'].mean()
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)
    return study.best_params